In [1]:
import torch

from torch.utils.data import DataLoader
from torch_geometric.utils import to_dense_adj

In [2]:
from helpers import CVFConfigDataset

In [3]:
class VanillaGNNLayer(torch.nn.Module):
    def __init__(self, dim_in, dim_out):
        super().__init__()
        self.linear = torch.nn.Linear(dim_in, dim_out, bias=False)

    def forward(self, x, adjacency):
        x = self.linear(x)
        # x = torch.sparse.mm(adjacency, x)
        x = torch.matmul(adjacency, x)
        return x

In [4]:
dataset = CVFConfigDataset("graph_1_config_rank_dataset.csv", "graph_1_edge_index.json")
data_loader = DataLoader(dataset, batch_size=1024, shuffle=False)

In [5]:
adjacency = to_dense_adj(dataset.edge_index.t().contiguous())[0]
adjacency += torch.eye(len(adjacency))
adjacency = adjacency.unsqueeze(0)
adjacency

tensor([[[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 1.]]])

In [ ]:
class VanillaGNN(torch.nn.Module):
    def __init__(self, dim_in, dim_h, dim_out):
        super().__init__()
        self.gnn1 = VanillaGNNLayer(dim_in, dim_h)
        self.gnn2 = VanillaGNNLayer(dim_h, dim_out)
        # self.out = torch.nn.Linear(dim_h, dim_out)

    def forward(self, x, adjacency):
        h = self.gnn1(x, adjacency)
        h = torch.relu(h)
        h = self.gnn2(h, adjacency)
        h = torch.relu(h)
        h = h.squeeze(-1)
        return h

    def fit(self, data_loader, epochs):
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.parameters(), lr=0.01, weight_decay=5e-4)
        self.train()
        for epoch in range(epochs + 1):
            avg_loss = 0
            count = 0
            for batch in data_loader:
                x = batch[0]
                # print(x, x.shape)
                y = batch[1]
                optimizer.zero_grad()
                out = self(x, adjacency)
                # print("output", out, "y", y)
                loss = criterion(out, y)
                avg_loss += loss
                count += 1
                loss.backward()
                optimizer.step()

            print("Loss:", avg_loss / count)

In [7]:
gnn = VanillaGNN(1, 64, 1)
print(gnn)
gnn.fit(data_loader, epochs=10)

VanillaGNN(
  (gnn1): VanillaGNNLayer(
    (linear): Linear(in_features=1, out_features=64, bias=False)
  )
  (gnn2): VanillaGNNLayer(
    (linear): Linear(in_features=64, out_features=1, bias=False)
  )
)
Loss: tensor(2.3026, grad_fn=<DivBackward0>)
Loss: tensor(2.3026, grad_fn=<DivBackward0>)
Loss: tensor(2.1652, grad_fn=<DivBackward0>)
Loss: tensor(0.5564, grad_fn=<DivBackward0>)
Loss: tensor(1.0782, grad_fn=<DivBackward0>)
Loss: tensor(1.2369, grad_fn=<DivBackward0>)
Loss: tensor(1.0293, grad_fn=<DivBackward0>)
Loss: tensor(0.7427, grad_fn=<DivBackward0>)
Loss: tensor(0.7874, grad_fn=<DivBackward0>)
Loss: tensor(0.6350, grad_fn=<DivBackward0>)
Loss: tensor(0.7285, grad_fn=<DivBackward0>)


In [8]:
for batch in data_loader:
    x = batch[0]
    y = batch[1]
    predicted = gnn(x, adjacency).argmax(dim=1)
    print("y", y, "predicted", predicted)

y tensor([9, 8, 8,  ..., 8, 8, 9]) predicted tensor([0, 0, 0,  ..., 0, 0, 0])
y tensor([0, 0, 0,  ..., 0, 0, 0]) predicted tensor([0, 0, 0,  ..., 0, 0, 0])
y tensor([0, 0, 0,  ..., 0, 0, 0]) predicted tensor([0, 0, 0,  ..., 0, 0, 0])
y tensor([0, 0, 0,  ..., 0, 0, 0]) predicted tensor([0, 0, 0,  ..., 0, 0, 0])
y tensor([0, 0, 0,  ..., 0, 0, 0]) predicted tensor([0, 0, 0,  ..., 0, 0, 0])
